# 4.5 Data Consistency Checks

## This script contains the following points:

### 01. Importing Libraries
### 02. Importing Data
### 03. Consistency Checks on products.csv
### 04. Consistency Checks on orders_wrangled.csv
### 06. Exporting Data

# 01. Importing Libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

# 02. Importing Data

In [2]:
# Set project folder as a string
path = r'/Users/matthewjones/Documents/CareerFoundry/Data Immersion/Achievement 4/04-2024 Instacart Basket Analysis'

In [3]:
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [4]:
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

# 03. Consistency Checks on products.csv

In [5]:
# Checking the shape of df_prods
df_prods.shape

(49693, 5)

## Mixed Values

#### df_prods does not have any columns with mixed values

In [6]:
# Create a test dataframe
df_test = pd.DataFrame()

In [7]:
# Create a mixed-type column
df_test['mix'] = ['a', 'b', 1, True]

In [8]:
df_test['mix'].dtype

dtype('O')

In [9]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [10]:
# Check for mixed types
for col in df_test.columns.tolist():
  weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


In [11]:
# Change column datatype to string
df_test['mix'] = df_test['mix'].astype('str')

In [12]:
# Check for mixed types
for col in df_test.columns.tolist():
  weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

#### No mixed value columns any longer

## Missing Values

In [13]:
# Check for missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [14]:
# Create a subset of missing values only
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [15]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [16]:
# Create a subset of df_prods that excludes all rows where the the product_name is missing
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [17]:
df_prods_clean.shape

(49677, 5)

#### The resulting dataframe is 16 rows shorter than the original

## Outlier Price Values

In [18]:
# Summarize the dataframe's main characteristics
df_prods_clean['prices'].describe()

count    49677.000000
mean         9.993164
std        453.592708
min          1.000000
25%          4.100000
50%          7.100000
75%         11.100000
max      99999.000000
Name: prices, dtype: float64

In [19]:
df_prods_clean['prices'].value_counts().sort_values(ascending = False)

prices
2.5        470
5.3        458
6.2        451
2.6        447
5.4        444
          ... 
15.6         1
21.0         1
99999.0      1
14900.0      1
18.3         1
Name: count, Length: 242, dtype: int64

In [20]:
# Marking outlier values in 'prices' as missing
df_prods_clean.loc[df_prods_clean['prices'] > 100, 'prices'] = np.nan

In [21]:
# Check that the maximum of the 'prices' column makes sense
df_prods_clean['prices'].max()

25.0

## Duplicate Rows

In [22]:
# Create a subset of df_prods_clean that has duplicate rows
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [23]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [24]:
# Create a subset of df_prods_clean with no duplicate rows
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [25]:
df_prods_clean_no_dups.shape

(49672, 5)

#### The resulting dataframe is 5 rows shorter than the previous dataframe

# 04. Consistency Checks on orders_wrangled.csv

## 2. Describe df_ords

In [26]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


#### In the days_since_prior_order column there are missing values (3.21e+06, instead of 3.42e+06). And the minimum and maximum values in this column suggest that one customer went as little as 0 days between orders, and one customer went as long as 30 days between orders. I would investigate this column.

#### In the order_day_of_week and order_hour_of_day columns the values range from 0-6 and 0-23, respectively. This corresponds to 7 days of the week, and 24 hours of the day. I don't think these columns need to be investigated.

#### In the order_number column, the maximum value is 100, which is an outlier. We can tell because the mean is only 17, and the 75th percentile is 23. I would look into this.

## 3. Check for mixed-type data

In [27]:
# Check for mixed-type columns
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

#### No columns have mixed-type data

In [28]:
df_ords.dtypes

Unnamed: 0                  int64
order_id                    int64
user_id                     int64
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

## 5. Check for missing values

In [29]:
# Sum the number of missing values in each column
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

#### As we noted before, days_since_prior_order is the only column with missing values. This could be because if a customer is placing their first order, there wouldn't be a value for when they placed their prior order. But these rows should not be omitted.

## 6. Address the missing values

In [30]:
#Create a crosstab between the “days_since_prior_order” column and the “order_number” column
crosstab = pd.crosstab(df_ords['days_since_prior_order'], df_ords['order_number'], dropna = False)

In [31]:
#Save the crosstab to clipboard
crosstab.to_clipboard()

In [32]:
# Create subset of missing values
df_ords_nan = df_ords[df_ords['days_since_prior_order'].isnull() == True]

In [33]:
#Check the shape of this subset
df_ords_nan.shape

(206209, 7)

In [34]:
# Create boolean check for if the order_number is 1
df_ords_new_customer = df_ords_nan['order_number'] == 1

In [35]:
# Count how many rows passed the boolean check
df_ords_new_customer.value_counts()

order_number
True    206209
Name: count, dtype: int64

#### All rows that were missing values for the days_since_prior_order column also had a 1 in the order_number column. Meaning all of these users were new customers. This is confirmed by the crosstab that showed all missing values were listed under Order Number == 1

In [36]:
# Create a new column that checks if days_since_prior_order has a missing value
df_ords['new_customer'] = df_ords['days_since_prior_order'].isna()

In [37]:
df_ords.head(20)

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer
0,0,2539329,1,1,2,8,NaN,True
1,1,2398795,1,2,3,7,15.0,False
2,2,473747,1,3,3,12,21.0,False
3,3,2254736,1,4,4,7,29.0,False
4,4,431534,1,5,4,15,28.0,False
5,5,3367565,1,6,2,7,19.0,False
6,6,550135,1,7,1,9,20.0,False
7,7,3108588,1,8,1,14,14.0,False
8,8,2295261,1,9,1,16,0.0,False
9,9,2550362,1,10,4,8,30.0,False


In [38]:
df_ords['new_customer'].value_counts()

new_customer
False    3214874
True      206209
Name: count, dtype: int64

#### Without overwriting the missing values in the days_since_prior_order column, I created a column to identify if this is a new customer or not. This will account for all of those appropriately missing values.

## 7. Check for duplicate rows

In [39]:
# Create a subset of df_ords with all duplicate rows
df_ords_dups = df_ords[df_ords.duplicated()]

In [40]:
df_ords_dups

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer


In [45]:
df_ords.shape

(3421083, 8)

#### There are no duplicate rows in the dataframe. If there were, I would create a subset of the df_ords dataframe by dropping the duplicates.

# 05. Exporting Data

In [41]:
# Export df_prods_clean_no_dups to the Prepared Data folder
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_cleaned.csv'))

In [42]:
# Export df_ords to the Prepared Data folder
df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_cleaned.csv'))